[Reference](https://medium.com/@datajournal/google-maps-scraping-guide-510dbcca4e92)

# Step 1: Setting Up Your Environment

Create a Project Directory

```
mkdir google-maps-scraper
cd google-maps-scraper
```

Set Up a Virtual Environment
```
python -m venv env
source env/bin/activate # On Windows, use `env\Scripts\activate`
```
Install Required Libraries
```
pip install selenium
```

In [2]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 10.1 MB/s eta 0:00:00


# Step 2: Configuring Selenium

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument(" - headless") # Run browser in the background
driver = webdriver.Chrome(service=Service(), options=options)
driver.get("https://www.google.com/maps")

# Step 3: Navigating the Google Maps Page

In [5]:
# Handle GDPR Prompt
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
try:
    accept_button = driver.find_element(By.CSS_SELECTOR, "[aria-label='Accept all']")
    accept_button.click()
except NoSuchElementException:
    print("No GDPR requirements detected")

In [7]:
# Submit a Search Query
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
search_box = WebDriverWait(driver, 5).until(
EC.presence_of_element_located((By.CSS_SELECTOR, "#searchboxinput"))
)
search_box.send_keys("Italian restaurants")
search_button = driver.find_element(By.CSS_SELECTOR, "button[aria-label='Search']")
search_button.click()

# Step 4: Extracting Business Data

In [8]:
business_items = WebDriverWait(driver, 10).until(
EC.presence_of_all_elements_located((By.XPATH, '//div[@role="feed"]//div[contains(@jsaction, "mouseover:pane")]'))
)

## Extract Basic Details

In [9]:
for item in business_items:
    name = item.find_element(By.CSS_SELECTOR, "div.fontHeadlineSmall").text
    link = item.find_element(By.CSS_SELECTOR, "a[jsaction]").get_attribute("href")
print(f"Business: {name}, Link: {link}")

## Extract Reviews and Ratings

In [10]:
import re
reviews_element = item.find_element(By.CSS_SELECTOR, "span[role='img']")
reviews_text = reviews_element.get_attribute("aria-label")
match = re.match(r"(\d+\.\d+) stars (\d+[,]*\d+) Reviews", reviews_text)
if match:
    stars = float(match.group(1))
    review_count = int(match.group(2).replace(",", ""))
print(f"Stars: {stars}, Reviews: {review_count}")

## Extract Additional Information

In [11]:
info_div = item.find_element(By.CSS_SELECTOR, ".fontBodyMedium")
spans = info_div.find_elements(By.XPATH, ".//span[not(@*) or @style]")
details = [span.text for span in spans if span.text.strip()]
print("Details:", details)

# Step 5: Saving Data to CSV

## Prepare Data for Export

In [12]:
data = []
for item in business_items:
    # Collect data as shown above and append to a list
    data.append({
    "name": name,
    "link": link,
    "stars": stars,
    "review_count": review_count,
    "details": "; ".join(details),
    })

## Write to CSV

In [13]:
import csv
with open("business_data.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()
    # writer.writerows(data)